In [ ]:
# Read Me:
### Analysis of THD Online Sales/Website metrics
### Regular Report will be based on rolling 13 week period

In [11]:
# Analysis of THD Online Sales by Online SKU
### Load in Correct Data and just reorganize it
import sys
sys.path.append(r'C:\Users\jmurillo\AppData\Local\Programs\Python\Python310\Lib\site-packages')


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn
from bs4 import BeautifulSoup
import dataframe_image as dfi
from plotnine import *
import win32com.client as win32
# from sklearn.linear_model import LinearRegression # Linear Regression Model
# from sklearn.preprocessing import StandardScaler #Z-score variables
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score #model evaluation
import re
from matplotlib.pyplot import figure
# import altair as alt
# import datum
import hvplot.pandas
# from vega_datasets import data as vds
import panel as pn
pn.extension('tabulator')



In [12]:
# Reformat
internet_metrics = pd.read_excel(r"C:\Users\jmurillo\Desktop\Misc\THD_Internet_Metrics_mktg\IndividualFiles\InternetRatingsReviews.xlsx")
internet_metrics.head(8)


# Clean up VendorDrill formatting for df 1
internet_metrics = internet_metrics.iloc[7:]
internet_metrics.columns = internet_metrics.iloc[0]
internet_metrics.drop(index=internet_metrics.index[0], 
                  axis=0, 
                  inplace=True)


# Drop columns that contain NaN
# internet_metrics = internet_metrics.iloc[:, :-3]

# Drop last row (totals row from VendorDrill Report)
internet_metrics = internet_metrics.drop(internet_metrics.index[len(internet_metrics)-1])


# Change Dtypes/replace NA's with 0
internet_metrics.update(internet_metrics[['Online Count of 1 Star Reviews +','Online Count of 2 Star Reviews +','Online Count of 3 Star Reviews +',
                                          'Online Count of 4 Star Reviews +','Online Count of 5 Star Reviews +','Online PIP Visits +',
                                          'Online PIP Conversion Rate +','Online Product Interaction Visits +','Online Gross Demand $ +',
                                          'Online Order Units +','Online Sales $ +','Online Cancel Units +','Online Cancel $ +',
                                          'Online Return $ +','Online Return Units +', 'is_multi_count','Week','Week_1','Year_1']].fillna(0))

internet_metrics = internet_metrics.astype({'Online Count of 1 Star Reviews +':'int64','Online Count of 2 Star Reviews +':'int64','Online Count of 3 Star Reviews +':'int64',
                                          'Online Count of 4 Star Reviews +':'int64','Online Count of 5 Star Reviews +':'int64','Online PIP Visits +':'int64',
                                          'Online PIP Conversion Rate +':'float64','Online Product Interaction Visits +':'int64','Online Gross Demand $ +':'float64',
                                          'Online Order Units +':'int64','Online Sales $ +':'float64','Online Cancel Units +':'int64','Online Cancel $ +':'float64',
                                          'Online Return $ +':'float64','Online Return Units +':'int64','Online Avg Rating  +':'float64',
                                           'Online Current List Price $ +':'float64','is_multi_count':'int64'})

# Convert Date to DateTime
internet_metrics['DateTime_Start']= pd.to_datetime(internet_metrics['DateTime_Start'])
internet_metrics['DateTime_End']= pd.to_datetime(internet_metrics['DateTime_End'])
#Reformat month year to month year
internet_metrics['Month_Year']= pd.to_datetime(internet_metrics['Month_Year'])
internet_metrics['Month_Year'] = internet_metrics['Month_Year'].dt.strftime('%m/%Y')
internet_metrics['Month_Year']= pd.to_datetime(internet_metrics['Month_Year'])

internet_metrics.columns = internet_metrics.columns.str.replace(' ','_')


In [13]:
# Find top 10 selling SKU's
group_by_sum = pd.DataFrame(internet_metrics.groupby(['Online_THD_SKU+'])['Online_Sales_$_+'].sum())
group_by_sum = group_by_sum.reset_index()
group_by_sum = group_by_sum.sort_values(by=['Online_Sales_$_+'], ascending=False)
group_by_sum.head(10)

,Online_THD_SKU+,Online_Sales_$_+
262,431429-SG APC 128OZ,950860.70
268,883387-SIMPLE GREEN APC 320OZ,640068.61
177,1002075713-SMPL GRN OUTDR ODOR ELIMINATOR 128OZ,587963.65
266,853534-SG PRO HEAVY DUTY 128OZ,326285.36
267,854029-SG PRO3PLUS ANTIBAC&DISINFECT 128OZ,289583.07
263,435909-SG APC CONCEN SPY 32OZ,210136.22
0,1000017290-SG APC 640OZ,207473.63
186,1002332519-5 GAL. EXTREME AIRCRAFT AND PRECISIO,178705.94
176,1002075704-SMPL GRN OUTDR ODOR ELIMINATOR 32OZ,167359.80
168,1001700777-1 GAL. CONCENTRATED ALL-PURPOSE CLEA,142998.24


In [14]:
# Select top 10 online selling items
top_10 = internet_metrics[(internet_metrics['Online_THD_SKU+'].str.contains("431429-SG APC 128OZ")==True) | 
                       (internet_metrics['Online_THD_SKU+'].str.contains("883387-SIMPLE GREEN APC 320OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("1002075713-SMPL GRN OUTDR ODOR ELIMINATOR 128OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("853534-SG PRO HEAVY DUTY 128OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("854029-SG PRO3PLUS ANTIBAC&DISINFECT 128OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("435909-SG APC CONCEN SPY 32OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("1000017290-SG APC 640OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("1002332519-5 GAL. EXTREME AIRCRAFT AND PRECISIO")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("1002075704-SMPL GRN OUTDR ODOR ELIMINATOR 32OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("1001700777-1 GAL. CONCENTRATED ALL-PURPOSE CLEA")==True)]

# Select top 5 online selling items
top_5 = internet_metrics[(internet_metrics['Online_THD_SKU+'].str.contains("431429-SG APC 128OZ")==True) | 
                       (internet_metrics['Online_THD_SKU+'].str.contains("883387-SIMPLE GREEN APC 320OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("1002075713-SMPL GRN OUTDR ODOR ELIMINATOR 128OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("853534-SG PRO HEAVY DUTY 128OZ")==True)|
                       (internet_metrics['Online_THD_SKU+'].str.contains("854029-SG PRO3PLUS ANTIBAC&DISINFECT 128OZ")==True)]
print(top_5.shape,top_10.shape)

(1522, 27) (3168, 27)


In [15]:
# plot top 10 over all time

#sn.set_palette("colorblind")
#ax = sn.lineplot(x = "DateTime_End", y = "Online_PIP_Visits_+", hue = "Online_THD_SKU+", data = top_10)
#plt.show()

In [16]:
# Interactive chart with drop-down menu all time by week end date
all_time_visits = top_10.hvplot(x='DateTime_End', y='Online_PIP_Visits_+', groupby='Online_THD_SKU+', kind='line')
all_time_visits

:DynamicMap   [Online_THD_SKU+]
   :Curve   [DateTime_End]   (Online_PIP_Visits_+)

In [17]:
# Interactive chart with drop-down menu all time by average grouped by month
# Still top 10 selling items
month_avg = pd.DataFrame(top_10.groupby(['Month_Year','Online_THD_SKU+'])['Online_PIP_Visits_+'].mean())


month_avg = month_avg.reset_index()
month_avg.rename(columns={"Online_PIP_Visits_+": "Online_PIP_Visits_+_monthly_avg"}, inplace = True)

month_avg.tail()
month_avg = month_avg.sort_values(by=['Month_Year'], ascending=True)

month_avg_visits = month_avg.hvplot(x='Month_Year', y='Online_PIP_Visits_+_monthly_avg', groupby='Online_THD_SKU+', kind='line')
month_avg_visits
# This plot gives me a better idea month to month what the goal would be for PIP visits for the top 10 SKU's
# you can also see the seasonality... the last half of the graph shows the later months for all years 2017-2022. They're significantly lower than the middle or first part of the graph
# If you visualize the graph as a loop... you can see that sales go down for outdoor eliminator during the fall/winter months... then pick back up in January into spring/summer.

:DynamicMap   [Online_THD_SKU+]
   :Curve   [Month_Year]   (Online_PIP_Visits_+_monthly_avg)

In [18]:
# Group by month and sum sales and canceled orders
#month_sum = pd.DataFrame(top_10.groupby(['Year_1'])[['Online_Sales_$_+','Online_Cancel_$_+']].sum())
#month_sum = month_sum.reset_index()
# Plot that shit
# multiple line plots

#plt.plot( 'Year_1', 'Online_Sales_$_+', data=month_sum, marker='', color='skyblue', linewidth=2, label = "Online Sales")
#plt.plot( 'Year_1', 'Online_Cancel_$_+', data=month_sum, marker='', color='red', linewidth=2, linestyle='dashed', label="Online Cancellations")

# show legend
#plt.legend()

# show graph
#plt.show()

### ------------------------------------------- Save Interactive Charts and Create Dashboard -------------------------------------------

In [19]:
# Save interactive chart as HTML for functionality
#### from bokeh.resources import INLINE
#### hvplot.save(zoink_plot, 'OnlinePIP_Visits_Top10_SKUs_THD.html', resources=INLINE)
os.chdir(r"C:\Users\jmurillo\Desktop\Misc\THD_Internet_Metrics_mktg\IndividualFiles")
#Layout using Template
template = pn.template.FastListTemplate(
    title='Internet Metrics - The Home Depot', 
    sidebar=[pn.pane.Markdown("# The Home Depot Online Sales"), 
             pn.pane.Markdown("#### This Dashboard displays summary statistics from Home Depot's Online Sales Performance. The following is data pulled directly from HomeDepot/Vendor Drill, something we were previously lacking insight on. We have great data on how we sell our products to them, but now how our products perform in their stores/on HomeDepot.com"), 
             pn.pane.PNG('Logo_SimpleGreen_White_Outline-Green.png', sizing_mode='scale_both'),
             pn.pane.PNG('TheHomeDepot.png', sizing_mode='scale_both')],
    main=[pn.Row(pn.Column(month_avg_visits, all_time_visits))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

### ------------------------------------------- Dashboard building -------------------------------------------

In [153]:
# Create some widgets (year slider)
# year_slider = pn.widgets.IntSlider(name = 'Year Slider', start = 2016, end = 2023, step = 1, value = 2016)
# year_slider

IntSlider(end=2023, name='Year Slider', start=2016, value=2016)

In [9]:

#top_10.columns = top_10.columns.str.replace('+','')
#top_10['Online_THD_SKU'].unique()
#skus = ['1000017290-SG APC 640OZ',
#       '1001700777-1 GAL. CONCENTRATED ALL-PURPOSE CLEA',
#       '1002075704-SMPL GRN OUTDR ODOR ELIMINATOR 32OZ',
#       '1002075713-SMPL GRN OUTDR ODOR ELIMINATOR 128OZ',
#       '1002332519-5 GAL. EXTREME AIRCRAFT AND PRECISIO',
#       '431429-SG APC 128OZ', '435909-SG APC CONCEN SPY 32OZ',
#       '853534-SG PRO HEAVY DUTY 128OZ',
#       '854029-SG PRO3PLUS ANTIBAC&DISINFECT 128OZ',
#       '883387-SIMPLE GREEN APC 320OZ']

#top_10 = top_10.astype({'Year_1':'int64'})
#top_10.head(3)


In [10]:
# First chart pipeline
#idf = top_10.interactive()
#total_pipeline = (
#    idf[
#        (idf.Year_1 <= year_slider) &
#        (idf.Online_THD_SKU.isin(skus))
#    ]
#    .groupby(['Online_THD_SKU', 'Month_Year'])['Online_PIP_Visits_'].mean()
#    .to_frame()
#    .reset_index()
#    .sort_values(by='Month_Year')  
#    .reset_index(drop=True)
#)

#total_pipeline

In [11]:
#total_plot = total_pipeline.hvplot(x = 'Month_Year', by = 'Online_THD_SKU', y = 'Online_PIP_Visits_', title = 'Online PIP Visits Per Year by SKU')
#total_plot